<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/feature_extractor_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install tsfresh
# getml --upgrade
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' #100 files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 400hz


In [ ]:
hzs=[100, 400, 800]
percents=[20, 50]
files_count=[10, 100, 1000]

In [ ]:
def file_downloader(file_name,file_var):
    file_var.to_csv(file_name, sep=',')
    print (f'Downloading {file_name}')
    files.download(file_name)

In [ ]:
def read_file(filename):
  return pd.read_csv(filename, sep=',')
def file_reader(hz,percent,files_count):
  train_feature = pd.DataFrame(read_file(f'{hz}hz-train-feature-{percent}percent-{files_count}files.csv'))
  test_feature = pd.DataFrame(read_file(f'{hz}hz-test-feature-{percent}percent-{files_count}files.csv'))
  train_target = pd.DataFrame(read_file(f'{hz}hz-train-target-{percent}percent-{files_count}files.csv'))
  test_target = pd.DataFrame(read_file(f'{hz}hz-test-target-{percent}percent-{files_count}files.csv'))

  train_feature = train_feature.drop(columns=['Unnamed: 0', 'Datum', 'P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])
  test_feature = test_feature.drop(columns=['Unnamed: 0', 'Datum', 'P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])
  train_target = train_target.drop(columns=['Unnamed: 0'])['color']
  test_target = test_target.drop(columns=['Unnamed: 0'])['color']

  return train_feature, test_feature, train_target, test_target

In [ ]:
for hz in hzs:
  for file in files_count:
    if file == 10 or file == 100:
      percent = 50
    else:
      percent = 20
    print (f'working with -----  hz: {hz}, percent: {percent}, files: {file}')

    train_feature, test_feature, train_target, test_target = file_reader(hz,percent,file)

    print("Extracting features from train part")
    extracted_features_train =  extract_features(
        train_feature,
        column_id="fileno",
        column_sort="Uhrzeit",
        impute_function=impute
    )

    print("Extracting features from test part")
    extracted_features_test = extract_features(
        test_feature,
        column_id="fileno",
        column_sort="Uhrzeit",
        impute_function=impute
    )

    feature_file_train = f'extracted-{hz}hz-train-feature-{percent}percent-{file}files.csv'
    target_file_train = f'extracted-{hz}hz-train-target-{percent}percent-{file}files.csv'
    feature_file_test = f'extracted-{hz}hz-test-feature-{percent}percent-{file}files.csv'
    target_file_test = f'extracted-{hz}hz-test-target-{percent}percent-{file}files.csv'

    file_downloader(feature_file_train,extracted_features_train)
    file_downloader(target_file_train,train_target)
    file_downloader(feature_file_test,extracted_features_test)
    file_downloader(target_file_test,test_target)




    # extracted_features_train.to_csv(feature_file_train, sep=',')
    # train_target.to_csv(target_file_train, sep=',')
    # extracted_features_test.to_csv(feature_file_test, sep=',')
    # test_target.to_csv(target_file_test, sep=',')
    # files.download(feature_file_train)
    # files.download(target_file_train)
    # files.download(feature_file_test)
    # files.download(target_file_test)